# Deeplab v3


In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from keras.layers import *

import nibabel as nib
from nilearn import plotting
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

import mia_datasets_tf as mia_data
import mia_evaluation as mia_eval
import mia_losses_tf as mia_losses
import mia_utils
import SimpleITK as sitk
import os 
import shutil
import inspect
import random

sitk.ProcessObject_SetGlobalWarningDisplay(False)
tf.config.run_functions_eagerly(True)

In [ ]:
model_directory = r"/ssd2/jupyter/MIA/deeplabv3"

if not os.path.exists(model_directory):
    os.makedirs(model_directory)

## Utilities

In [ ]:
def run_training(model_creator, fit_params, num_runs, directory, name):
    best_val_loss = float('inf')
    best_model_path = None

    val_losses = []

    valid_fit_args = inspect.signature(tf.keras.Model.fit).parameters.keys()
    filtered_fit_params = {key: value for key, value in fit_params.items() if key in valid_fit_args}

    for run in range(num_runs):
        print(f"Training run {run + 1}/{num_runs}\n")
        model = model_creator()

        run_model_path = os.path.join(directory, f"{name}_{run}.keras")

        checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(filepath=run_model_path, monitor='val_loss', save_best_only=True, mode='min')

        if "callbacks" in filtered_fit_params:
            filtered_fit_params['callbacks'].append(checkpoint_cb)
        else:
            filtered_fit_params['callbacks'] = [checkpoint_cb]
    
        history = model.fit(
            fit_params['training_data'],
            **filtered_fit_params)

        val_loss = min(history.history['val_loss'])
        val_losses.append(history.history['val_loss'])

        print(f"\nRun {run + 1} best val_loss = {val_loss:.6f}")

        if val_loss < best_val_loss:
            print(f"New best model found")
            best_val_loss = val_loss
            best_model_path = run_model_path

    print(f"\n Best model: {best_model_path} with val_loss = {best_val_loss:.6f}")

    return best_model_path, val_losses

In [ ]:
def plot_side_by_side(images : list[np.ndarray], titles : list[str] | None):
    fig, axs = plt.subplots(1, len(images), figsize=(15, 5))
    
    for index, image in enumerate(images):
        axs[index].imshow(image[image.shape[0] // 2])
        if titles is not None:
            axs[index].set_title(titles[index])
        axs[index].axis('off')

    plt.tight_layout()

In [ ]:
def show_history(history, y_lim=(0.0, 1.5)):
    plt.figure(figsize=(10, 6))
    for i, run_val_loss in enumerate(history):
        plt.plot(run_val_loss, label=f'Run {i + 1}')

    plt.title('Validation Loss Across Runs')
    plt.xlabel('Epochs')
    plt.ylabel('Validation Loss')
    plt.ylim(y_lim)
    plt.legend()
    plt.show()

In [ ]:
def seed_all(seed=2141):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

In [ ]:
colors = [(255, 60, 60), (140, 255, 140), (140, 200, 255)] 
custom_cmap = ListedColormap(colors)

In [ ]:
slicer = mia_data.create_slicer(0)

In [ ]:
def plot_slices(dataset, row_size):
    num_slices = len(dataset)
    num_cols = min(8, num_slices)
    num_rows = (num_slices + num_cols - 1) // num_cols

    plt.figure(figsize=(8, num_rows * row_size))
    for i, slice_data in enumerate(dataset):
        plt.subplot(num_rows, num_cols, i + 1)
        plt.imshow(slice_data[0].numpy(), cmap='gray')  
        plt.axis('off') 
        plt.title(f"Slice {i + 1}") 
   
    plt.tight_layout()
    plt.show()

## Model

In [ ]:
def conv_block(input_layer, num_filters, kernel_size=3, strides=1, dilation_rate=1,
                use_bias=False, padding='same', activation='relu', name=""):
        layer = Conv2D(filters=num_filters, kernel_size=kernel_size, strides=strides, 
                       dilation_rate=dilation_rate, use_bias=use_bias, padding=padding,
                         kernel_initializer="he_normal", name=f"{name}_conv")(input_layer)
        layer = BatchNormalization(name=f"{name}_norm")(layer)
        if activation:
            layer = Activation(activation, name=f"{name}_acti")(layer)
        return layer

def encoder_block(input_layer, filters, name="", pooling=True):
    layer = input_layer
    for i, f in enumerate(filters):
        layer = conv_block(layer, num_filters=f, name=f'{name}_block{i}')
    if pooling:
        layer = MaxPooling2D((2, 2), name=f'{name}_maxpool')(layer)
    return layer

def aspp_block(input_layer, num_filters, name=""):
    dims = input_layer.shape

    out_pool = AveragePooling2D(pool_size=dims[-3:-1], name=f"{name}_avrg_pool")(input_layer)
    out_pool = conv_block(out_pool, num_filters=num_filters, kernel_size=1, use_bias=True, name=f"{name}_conv1")
    out_pool = UpSampling2D(size=dims[-3:-1], interpolation="bilinear", name=f"{name}_upsampl")(out_pool)

    out_1 = conv_block(input_layer, num_filters=num_filters, kernel_size=1, dilation_rate=1, name=f"{name}_conv2")
    out_4 = conv_block(input_layer, num_filters=num_filters, kernel_size=3, dilation_rate=4, name=f"{name}_conv3")
    out_8 = conv_block(input_layer, num_filters=num_filters, kernel_size=3, dilation_rate=8, name=f"{name}_conv4")

    layer = Concatenate(axis=-1, name=f"{name}_concat")([out_pool, out_1, out_4, out_8])
    output_layer = conv_block(layer, num_filters=num_filters, kernel_size=1, name=f"{name}_conv5")

    return output_layer    

def deeplabv3_compiled(input_size, n_classes=1, learning_rate=0.0001, enc_filters=32, aspp_filters=256):
    inputs = Input(input_size[-3:], name='inputs')

    encoder_1 = encoder_block(inputs, filters=(enc_filters, enc_filters), name="enc_1")
    encoder_2 = encoder_block(encoder_1, filters=(enc_filters * 2, enc_filters * 2), name="enc_2")
    encoder_3 = encoder_block(encoder_2, filters=(enc_filters * 4, enc_filters * 4), name="enc_3")

    aspp = aspp_block(encoder_3, num_filters=aspp_filters, name="aspp")
    dec_input_a = UpSampling2D(size=(input_size[0] // aspp.shape[-3] // 2, input_size[1] // aspp.shape[-2] // 2),
                                interpolation="bilinear", name="dec_input_a")(aspp)

    dec_input_b = conv_block(encoder_1, num_filters=64, kernel_size=1, name="dec_input_b")

    layer = Concatenate(axis=-1, name="dec_concat")([dec_input_a, dec_input_b])
    layer = conv_block(layer, num_filters=128, kernel_size=3, name=f"dec_conv")
    layer = UpSampling2D(size=(input_size[0] // layer.shape[-3], input_size[1] // layer.shape[-2]),
                          interpolation="bilinear", name="dec_output")(layer)

    outputs = conv_block(layer, num_filters=n_classes, kernel_size=1, activation='Softmax', name="outputs")

    model = keras.Model(inputs=inputs, outputs=outputs, name="DeepLabV3")
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                   loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=[keras.metrics.SparseCategoricalAccuracy(name="sparse_categorical_accuracy")])

    return model

## Iseg 2019

In [ ]:
iseg2019_training = mia_data.Iseg2019Processed(r'/ssd2/jupyter/MIA/split_datasets/iseg2019_ns/training')
iseg2019_validation = mia_data.Iseg2019Processed(r'/ssd2/jupyter/MIA/split_datasets/iseg2019_ns/validation')
iseg2019_testing = mia_data.Iseg2019Processed(r'/ssd2/jupyter/MIA/split_datasets/iseg2019_ns/testing')

In [ ]:
dataset_specific_directory = os.path.join(model_directory, "iseg2019_ns")
if not os.path.exists(dataset_specific_directory):
    os.makedirs(dataset_specific_directory)

inference_directory = os.path.join(dataset_specific_directory, "inference")
if not os.path.exists(inference_directory):
    os.makedirs(inference_directory)

labels_dictionary = {"CSF" : 1, "GM" : 2, "WM" : 3}

In [ ]:
t1 = iseg2019_training.subjects[0].get_T1()
t2 = iseg2019_training.subjects[0].get_T2()
label = iseg2019_training.subjects[0].get_label()

plot_side_by_side([sitk.GetArrayViewFromImage(t1), sitk.GetArrayViewFromImage(t2), sitk.GetArrayViewFromImage(label)], ["T1", "T2", "Label"] )

In [ ]:
testing = mia_data.Iseg2019Processed('/ssd2/jupyter/MIA/split_datasets/iseg2019_ns/testing').T2_dataset()

input_shape = next(testing.take(1).as_numpy_iterator())[0].shape
input_shape

In [ ]:
sliced_example = testing.take(1).flat_map(mia_data.create_slicer(0))
plot_slices([x for x in sliced_example], 1)

In [ ]:
sliced_example = testing.take(1).flat_map(mia_data.create_slicer(1))
plot_slices([x for x in sliced_example], 1)

In [ ]:
sliced_example = testing.take(1).flat_map(mia_data.create_slicer(2))
plot_slices([x for x in sliced_example], 1)

### T1 + T2

#### Setup

In [ ]:
model_file = os.path.join(dataset_specific_directory, "iseg2019_T1T2.keras")

training_cropped = iseg2019_training.T1_T2_dataset()
validation_cropped = iseg2019_validation.T1_T2_dataset()
testing = iseg2019_testing.T1_T2_dataset()

batch_size = 128
num_channels = 2
training_cropped = training_cropped.flat_map(slicer).shuffle(512).batch(batch_size)
validation_cropped = validation_cropped.flat_map(slicer).shuffle(512).batch(batch_size)

slices = next(training_cropped.take(1).as_numpy_iterator())

input_shape = (slices[0].shape[1:])
input_shape

#### Training

In [ ]:
deeplab_creator = lambda : deeplabv3_compiled(input_size=input_shape, n_classes=len(labels_dictionary) + 1, learning_rate=0.0001)

callbacks_list = [
    keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=50),
]

fit_params = {
    "callbacks" : callbacks_list,
    "training_data" : training_cropped,
    "validation_data" : validation_cropped,
    'epochs' : 1000
}

seed_all()
best_model_path, history = run_training(deeplab_creator, fit_params, 5, dataset_specific_directory, "iseg2019_T1T2")

if os.path.exists(model_file):
    os.remove(model_file)
    
shutil.copy(best_model_path, model_file)

In [ ]:
show_history(history)

#### Inference

In [ ]:
deeplab = keras.saving.load_model(model_file)

In [ ]:
testing_slices  = testing.flat_map(slicer).batch(1)

prediction = deeplab.predict(testing_slices)
predicted_labels = prediction.argmax(axis=-1)

mia_utils.writeImagesArray(predicted_labels, inference_directory, lambda x : f"T1_T2{x}.nii.gz", lambda x : iseg2019_testing.subjects[x].get_label())

In [ ]:
mia_utils.interactive_display(predicted_labels, (1,3), "Iseg 2019 T1 + T2 Inference", cmap=custom_cmap)

In [ ]:
mia_utils.interactive_display(next(testing.take(1).as_numpy_iterator())[1], (1,3), title="Iseg 2019 T1+T2 Ground truth", cmap=custom_cmap)

In [ ]:
predicted_image = sitk.GetImageFromArray(predicted_labels)
truth_image = sitk.GetImageFromArray(testing.as_numpy_iterator().next()[1])

eval = mia_eval.evaluateImage(predicted_image, truth_image, labels_dictionary)

In [ ]:
mia_eval.createRecord("iseg2019", eval)

In [ ]:
tf.config.experimental.get_memory_info('GPU:0')

## BONBID-HIE 2023

In [ ]:
bondid2023_training = mia_data.BONDID2023Processed(r'/ssd2/jupyter/MIA/split_datasets/bonbid2023_ns/training', "nii.gz")
bondid2023_validation = mia_data.BONDID2023Processed(r'/ssd2/jupyter/MIA/split_datasets/bonbid2023_ns/validation',  "nii.gz")
bondid2023_testing = mia_data.BONDID2023Processed(r'/ssd2/jupyter/MIA/split_datasets/bonbid2023_ns/testing',  "nii.gz")

In [ ]:
dataset_specific_directory = os.path.join(model_directory, "bonbid2023_ns")
if not os.path.exists(dataset_specific_directory):
    os.makedirs(dataset_specific_directory)

inference_directory = os.path.join(dataset_specific_directory, "inference")
if not os.path.exists(inference_directory):
    os.makedirs(inference_directory)

labels_dictionary = {"Lesion" : 1}

In [ ]:
adc = bondid2023_training.subjects[0].get_ADC_ss()
z_adc = bondid2023_training.subjects[0].get_Z_ADC()
label = bondid2023_training.subjects[0].get_label()

plot_side_by_side([sitk.GetArrayViewFromImage(adc), sitk.GetArrayViewFromImage(z_adc), sitk.GetArrayViewFromImage(label)], ["ADC_ss", "Z_ADC", "Label"] )

In [ ]:
sliced_example = bondid2023_testing.ADC_ss_dataset().take(1).flat_map(mia_data.create_slicer(0))
plot_slices([x for x in sliced_example], 1)

### ADC + Z_ADC


#### Setup

In [ ]:
model_file = os.path.join(dataset_specific_directory, "bonbid2023_adc_zadc.keras")

training = bondid2023_training.ADC_ss_Z_ADC_dataset()
testing = bondid2023_testing.ADC_ss_Z_ADC_dataset()
validation = bondid2023_validation.ADC_ss_Z_ADC_dataset()

batch_size = 64

training = training.flat_map(slicer).batch(batch_size)
validation = validation.flat_map(slicer).batch(batch_size)

slices = next(training.take(1).as_numpy_iterator())

input_shape = slices[0].shape[1:]
input_shape

#### Training

In [ ]:
deeplab_creator = lambda : deeplabv3_compiled(input_size=input_shape, n_classes=len(labels_dictionary) + 1, learning_rate=0.0001)

callbacks_list = [
    keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=50),
    tf.keras.callbacks.ReduceLROnPlateau( monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6)
]

fit_params = {
    "callbacks" : callbacks_list,
    "training_data" : training,
    "validation_data" : validation,
    'epochs' : 1000
}

seed_all()
best_model_path, history = run_training(deeplab_creator, fit_params, 5, dataset_specific_directory, "bonbid2023_adc_zadc")

if os.path.exists(model_file):
    shutil.rmtree(model_file)
    
shutil.copy(best_model_path, model_file)

In [ ]:
show_history(history)

#### Inference

In [ ]:
deeplab = keras.saving.load_model(model_file)

In [ ]:
evaluations = []

for index, img in enumerate(testing):
    prediction = deeplab.predict(img[0])
    predicted_labels = tf.math.argmax(prediction, -1).numpy()

    metrics = mia_eval.evaluateImage(sitk.GetImageFromArray(predicted_labels), sitk.GetImageFromArray(img[1].numpy()), labels_dictionary)
    evaluations.append((str(index), metrics))
    
    mia_utils.writeImageArray(predicted_labels, 
                              os.path.join(inference_directory, f"ADC_Z_ADC_{index}.nii.gz"),
                               bondid2023_testing.subjects[index].get_label())


In [ ]:
mia_utils.interactive_display(predicted_labels, (0,1), "Bondid 2023 ADC + Z_ADC Inference", cmap=custom_cmap)

In [ ]:
mia_utils.interactive_display(sitk.GetArrayFromImage(bondid2023_testing.subjects[index].get_label()), (0,1), title="Bondid 2023 ADC + Z_ADC Ground truth", cmap=custom_cmap)

In [ ]:
mia_eval.createDataFrame(evaluations)

In [ ]:
tf.config.experimental.get_memory_info('GPU:0')

## TACR 6 HIE

In [ ]:
tacrhie_training = mia_data.TACRHIE6Dataset(r'/ssd2/jupyter/MIA/split_datasets/tacrhie/training', "nii.gz")
tacrhie_validation = mia_data.TACRHIE6Dataset(r'/ssd2/jupyter/MIA/split_datasets/tacrhie/validation',  "nii.gz")
tacrhie_testing = mia_data.TACRHIE6Dataset(r'/ssd2/jupyter/MIA/split_datasets/tacrhie/testing',  "nii.gz")

In [ ]:
target_shape = (128, 128, 128)

training_cropped = mia_data.CroppedDataset(tacrhie_training.aseg_dataset(), target_shape).dataset()
validation_cropped = mia_data.CroppedDataset(tacrhie_validation.aseg_dataset(), target_shape).dataset()
testing = mia_data.CroppedDataset(tacrhie_testing.aseg_dataset(), target_shape).dataset()

In [ ]:
sliced_example = training_cropped.take(1).flat_map(mia_data.create_slicer(0))
plot_slices([x for x in sliced_example], 1)

In [ ]:
import json

dataset_specific_directory = os.path.join(model_directory, "tacrhie", "aseg")
os.makedirs(dataset_specific_directory, exist_ok=True)

inference_directory = os.path.join(dataset_specific_directory, "inference")
os.makedirs(inference_directory, exist_ok=True)

with open(r'/ssd2/jupyter/MIA/split_datasets/tacrhie/aseg_labels.json') as file:
    labels_dictionary = json.load(file)
    del labels_dictionary['0']

    labels_dictionary = {v : float(k) for k,v in labels_dictionary.items()}

with open(r'/ssd2/jupyter/MIA/split_datasets/tacrhie/aseg_colors.json') as file:
    colours_dictionary = json.load(file)
    del colours_dictionary['0']
        
    custom_cmap = ListedColormap([np.array(colours_dictionary[str(k)][:3]) / 255.0 for k in sorted(int(a) for a in colours_dictionary.keys())])

In [ ]:
norm = tacrhie_training.subjects[0].get_norm()
aseg = tacrhie_training.subjects[0].get_aseg()
aseg_aparc = tacrhie_training.subjects[0].get_aseg_aparc()

plot_side_by_side([sitk.GetArrayViewFromImage(norm), sitk.GetArrayViewFromImage(aseg), sitk.GetArrayViewFromImage(aseg_aparc)], ["norm", "aseg", "aseg+aparc"] )

### aseg


#### Setup

In [ ]:
model_file = os.path.join(dataset_specific_directory, "tacrhie6_aseg.keras")

num_channels = 1
batch_size = 128

training = training_cropped.flat_map(slicer).batch(batch_size)
validation = validation_cropped.flat_map(slicer).batch(batch_size)

slices = next(training_cropped.take(1).as_numpy_iterator())

input_shape = (*slices[0].shape[1:], num_channels)
input_shape

#### Training

In [ ]:
deeplab_creator = lambda : deeplabv3_compiled(input_size=input_shape, n_classes=len(labels_dictionary) + 1, learning_rate=0.001, enc_filters=64, aspp_filters=512)

callbacks_list = [
    keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=50),
    tf.keras.callbacks.ReduceLROnPlateau( monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6)
]

fit_params = {
    "callbacks" : callbacks_list,
    "training_data" : training,
    "validation_data" : validation,
    'epochs' : 1000
}

seed_all()
best_model_path, history = run_training(deeplab_creator, fit_params, 5, dataset_specific_directory, "tacrhie_aseg")

if os.path.exists(model_file):
    os.remove(model_file)
    
shutil.copy(best_model_path, model_file)

In [ ]:
show_history(history)

#### Inference

In [ ]:
deeplab = keras.saving.load_model(model_file)

In [ ]:
evaluations = []

for index, img in enumerate(testing):
    prediction = deeplab.predict(img[0])
    predicted_labels = tf.math.argmax(prediction, -1).numpy()

    metrics = mia_eval.evaluateImage(sitk.GetImageFromArray(predicted_labels), sitk.GetImageFromArray(img[1].numpy()), labels_dictionary)
    evaluations.append((str(index), metrics))
    
    shape = (256, 256, 256)
    begin = (np.array(shape) - predicted_labels.shape) // 2
    begin[0] = 0
    result = mia_utils.embed_tensor(predicted_labels, shape, begin)

    mia_utils.writeImageArray(result, 
                              os.path.join(inference_directory, f"{tacrhie_testing.subjects[index].number}.nii.gz"),
                              tacrhie_testing.subjects[index].get_aseg())


In [ ]:
tf.config.experimental.get_memory_info('GPU:0')

In [ ]:
mia_utils.interactive_display(predicted_labels, (1,256), title="TACR-HIE aseg Inference", cmap=custom_cmap)

In [ ]:
truth_array = next(testing.take(1).as_numpy_iterator())[1]

mia_utils.interactive_display(truth_array, (1,256), title="TACR-HIE aseg Ground truth", cmap=custom_cmap)

In [ ]:
mia_eval.createDataFrame(evaluations)

In [ ]:
tf.config.experimental.get_memory_info('GPU:0')

### aseg + aparc

In [ ]:
target_shape = (128, 128, 128)

training_cropped = mia_data.CroppedDataset(tacrhie_training.aseg_aparc_dataset(), target_shape).dataset()
validation_cropped = mia_data.CroppedDataset(tacrhie_validation.aseg_aparc_dataset(), target_shape).dataset()
testing = mia_data.CroppedDataset(tacrhie_testing.aseg_aparc_dataset(), target_shape).dataset()

In [ ]:
import json

dataset_specific_directory = os.path.join(model_directory, "tacrhie", "aseg_aparc")
os.makedirs(dataset_specific_directory, exist_ok=True)

inference_directory = os.path.join(dataset_specific_directory, "inference")
os.makedirs(inference_directory, exist_ok=True)

with open(r'/ssd2/jupyter/MIA/split_datasets/tacrhie/aseg_aparc_labels.json') as file:
    labels_dictionary = json.load(file)
    del labels_dictionary['0']

    labels_dictionary = {v : float(k) for k,v in labels_dictionary.items()}

with open(r'/ssd2/jupyter/MIA/split_datasets/tacrhie/aseg_aparc_colors.json') as file:
    colours_dictionary = json.load(file)
    del colours_dictionary['0']
        
    custom_cmap = ListedColormap([np.array(colours_dictionary[str(k)][:3]) / 255.0 for k in sorted(int(a) for a in colours_dictionary.keys())])

In [ ]:
norm = tacrhie_training.subjects[0].get_norm()
aseg = tacrhie_training.subjects[0].get_aseg()
aseg_aparc = tacrhie_training.subjects[0].get_aseg_aparc()

plot_side_by_side([sitk.GetArrayViewFromImage(norm), sitk.GetArrayViewFromImage(aseg), sitk.GetArrayViewFromImage(aseg_aparc)], ["norm", "aseg", "aseg+aparc"] )

### aseg + aparc


#### Setup

In [ ]:
model_file = os.path.join(dataset_specific_directory, "tacrhie6_aseg_aparc.keras")

num_channels = 1
batch_size = 128

training_cropped = training_cropped.flat_map(slicer).batch(batch_size)
validation_cropped = validation_cropped.flat_map(slicer).batch(batch_size)

slices = next(training_cropped.take(1).as_numpy_iterator())

input_shape = (*slices[0].shape[1:], num_channels)
input_shape

#### Training

In [ ]:
deeplab_creator = lambda : deeplabv3_compiled(input_size=input_shape, n_classes=len(labels_dictionary) + 1, learning_rate=0.001, enc_filters=64, aspp_filters=512)

callbacks_list = [
    keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=50),
    tf.keras.callbacks.ReduceLROnPlateau( monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6)
]

fit_params = {
    "callbacks" : callbacks_list,
    "training_data" : training_cropped,
    "validation_data" : validation_cropped,
    'epochs' : 1000
}

seed_all()
best_model_path, history = run_training(deeplab_creator, fit_params, 5, dataset_specific_directory, "tacrhie_aseg_aparc")

if os.path.exists(model_file):
    os.remove(model_file)
    
shutil.copy(best_model_path, model_file)

In [ ]:
show_history(history)

#### Inference

In [ ]:
deeplab = keras.saving.load_model(model_file)

In [ ]:
evaluations = []

for index, img in enumerate(testing):
    prediction = deeplab.predict(img[0])
    predicted_labels = tf.math.argmax(prediction, -1).numpy()

    metrics = mia_eval.evaluateImage(sitk.GetImageFromArray(predicted_labels), sitk.GetImageFromArray(img[1].numpy()), labels_dictionary)
    evaluations.append((str(index), metrics))
    
    shape = (256, 256, 256)
    begin = (np.array(shape) - predicted_labels.shape) // 2
    begin[0] = 0
    result = mia_utils.embed_tensor(predicted_labels, shape, begin)

    mia_utils.writeImageArray(result, 
                              os.path.join(inference_directory, f"{tacrhie_testing.subjects[index].number}.nii.gz"),
                              tacrhie_testing.subjects[index].get_aseg())


In [ ]:
tf.config.experimental.get_memory_info('GPU:0')

In [ ]:
mia_utils.interactive_display(predicted_labels, (1, 256), title="TACR-HIE aseg+aparc Inference", cmap=custom_cmap)

In [ ]:
truth_array = next(testing.take(1).as_numpy_iterator())[1]

mia_utils.interactive_display(truth_array, (1, 256), title="TACR-HIE aseg+aparc  Ground truth", cmap=custom_cmap)

In [ ]:
mia_eval.createDataFrame(evaluations)

In [ ]:
tf.config.experimental.get_memory_info('GPU:0')